# Instructions To Run and Background 

This notebook represents the work done to execute **Deep Q-Learning** on the Flappy Bird enviornment. Due to the niche nature of the Flappy Bird game, assets/ folder is included as this loads the game assets, and the notebook will not be able to run without it.

The point of this notebook is demonstrate the training process as well as how to get started with testing an existing model. To begin, install the following: 
1. Have a Python 3.8.5 Environment Installed, as well as Tensorflow, Pygame. 


If you ever encounter the following error: `NotFoundError: Key Variable_10 not found in checkpoint` you must restart the notebook kernel as this is an error relating to a non-terminating tensorflow session. 

# First, we need to set up the PyGame and Flappy Bird enviornment. 
The source code for this is attributed to: https://github.com/sourabhv/FlapPyBird

The original game of Flappy Bird is actually taken down from the market, as the original creator felt it was too addicting. So, a clone must be used. 

In [1]:
import pygame
import sys

# disable rendering 
import os
os.environ['SDL_VIDEODRIVER']='dummy'

pygame.init()

def load():
    # path of player with different states
    PLAYER_PATH = (
            'assets/sprites/redbird-upflap.png',
            'assets/sprites/redbird-midflap.png',
            'assets/sprites/redbird-downflap.png'
    )

    # path of background
    BACKGROUND_PATH = 'assets/sprites/background-black.png'

    # path of pipe
    PIPE_PATH = 'assets/sprites/pipe-green.png'

    IMAGES, SOUNDS, HITMASKS = {}, {}, {}

    # numbers sprites for score display
    IMAGES['numbers'] = (
        pygame.image.load('assets/sprites/0.png').convert_alpha(),
        pygame.image.load('assets/sprites/1.png').convert_alpha(),
        pygame.image.load('assets/sprites/2.png').convert_alpha(),
        pygame.image.load('assets/sprites/3.png').convert_alpha(),
        pygame.image.load('assets/sprites/4.png').convert_alpha(),
        pygame.image.load('assets/sprites/5.png').convert_alpha(),
        pygame.image.load('assets/sprites/6.png').convert_alpha(),
        pygame.image.load('assets/sprites/7.png').convert_alpha(),
        pygame.image.load('assets/sprites/8.png').convert_alpha(),
        pygame.image.load('assets/sprites/9.png').convert_alpha()
    )

    # base (ground) sprite
    IMAGES['base'] = pygame.image.load('assets/sprites/base.png').convert_alpha()

    # sounds
    if 'win' in sys.platform:
        soundExt = '.wav'
    else:
        soundExt = '.ogg'
        
    # select random background sprites
    IMAGES['background'] = pygame.image.load(BACKGROUND_PATH).convert()

    # select random player sprites
    IMAGES['player'] = (
        pygame.image.load(PLAYER_PATH[0]).convert_alpha(),
        pygame.image.load(PLAYER_PATH[1]).convert_alpha(),
        pygame.image.load(PLAYER_PATH[2]).convert_alpha(),
    )

    # select random pipe sprites
    IMAGES['pipe'] = (
        pygame.transform.rotate(
            pygame.image.load(PIPE_PATH).convert_alpha(), 180),
        pygame.image.load(PIPE_PATH).convert_alpha(),
    )

    # hismask for pipes
    HITMASKS['pipe'] = (
        getHitmask(IMAGES['pipe'][0]),
        getHitmask(IMAGES['pipe'][1]),
    )

    # hitmask for player
    HITMASKS['player'] = (
        getHitmask(IMAGES['player'][0]),
        getHitmask(IMAGES['player'][1]),
        getHitmask(IMAGES['player'][2]),
    )

    return IMAGES, SOUNDS, HITMASKS

def getHitmask(image):
    """returns a hitmask using an image's alpha."""
    mask = []
    for x in range(image.get_width()):
        mask.append([])
        for y in range(image.get_height()):
            mask[x].append(bool(image.get_at((x,y))[3]))
    return mask


pygame 2.0.3 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# This code will allow for the Game State to be declared. 

import numpy as np
import sys
import random
import pygame
import pygame.surfarray as surfarray
from pygame.locals import *
from itertools import cycle

# disable rendering 
import os
os.environ['SDL_VIDEODRIVER']='dummy'

pygame.init()

# FPS set to 60 for fast computation. 
FPS = 60
SCREENWIDTH  = 288
SCREENHEIGHT = 512

FPSCLOCK = pygame.time.Clock()
SCREEN = pygame.display.set_mode((SCREENWIDTH, SCREENHEIGHT))
pygame.display.set_caption('Flappy Bird')

IMAGES, SOUNDS, HITMASKS = load()
PIPEGAPSIZE = 100 # gap between upper and lower part of pipe
BASEY = SCREENHEIGHT * 0.79

PLAYER_WIDTH = IMAGES['player'][0].get_width()
PLAYER_HEIGHT = IMAGES['player'][0].get_height()
PIPE_WIDTH = IMAGES['pipe'][0].get_width()
PIPE_HEIGHT = IMAGES['pipe'][0].get_height()
BACKGROUND_WIDTH = IMAGES['background'].get_width()

PLAYER_INDEX_GEN = cycle([0, 1, 2, 1])


class GameState:
    def __init__(self):
        self.score = self.playerIndex = self.loopIter = 0
        self.playerx = int(SCREENWIDTH * 0.2)
        self.playery = int((SCREENHEIGHT - PLAYER_HEIGHT) / 2)
        self.basex = 0
        self.baseShift = IMAGES['base'].get_width() - BACKGROUND_WIDTH

        # generating random pipes for the gameplay. 
        newPipe1 = getRandomPipe()
        newPipe2 = getRandomPipe()
        self.upperPipes = [
            {'x': SCREENWIDTH, 'y': newPipe1[0]['y']},
            {'x': SCREENWIDTH + (SCREENWIDTH / 2), 'y': newPipe2[0]['y']},
        ]
        self.lowerPipes = [
            {'x': SCREENWIDTH, 'y': newPipe1[1]['y']},
            {'x': SCREENWIDTH + (SCREENWIDTH / 2), 'y': newPipe2[1]['y']},
        ]

        self.pipeVelX = -4
        self.playerVelY    =  0    # player's velocity along Y, default same as playerFlapped
        self.playerMaxVelY =  10   # max vel along Y, max descend speed
        self.playerMinVelY =  -8   # min vel along Y, max ascend speed
        self.playerAccY    =   1   # players downward accleration
        self.playerFlapAcc =  -9   # players speed on flapping
        self.playerFlapped = False # True when player flaps

    def frame_step(self, input_actions):
        pygame.event.pump()

        reward = 0.1
        terminal = False

        if sum(input_actions) != 1:
            raise ValueError('Multiple input actions!')

        if input_actions[1] == 1:
            if self.playery > -2 * PLAYER_HEIGHT:
                self.playerVelY = self.playerFlapAcc
                self.playerFlapped = True

        # check for score
        playerMidPos = self.playerx + PLAYER_WIDTH / 2
        for pipe in self.upperPipes:
            pipeMidPos = pipe['x'] + PIPE_WIDTH / 2
            if pipeMidPos <= playerMidPos < pipeMidPos + 4:
                self.score += 1
                reward = 1

        # playerIndex basex change
        if (self.loopIter + 1) % 3 == 0:
            self.playerIndex = next(PLAYER_INDEX_GEN)
        self.loopIter = (self.loopIter + 1) % 30
        self.basex = -((-self.basex + 100) % self.baseShift)

        # player's movement
        if self.playerVelY < self.playerMaxVelY and not self.playerFlapped:
            self.playerVelY += self.playerAccY
        if self.playerFlapped:
            self.playerFlapped = False
        self.playery += min(self.playerVelY, BASEY - self.playery - PLAYER_HEIGHT)
        if self.playery < 0:
            self.playery = 0

        # move pipes to left
        for uPipe, lPipe in zip(self.upperPipes, self.lowerPipes):
            uPipe['x'] += self.pipeVelX
            lPipe['x'] += self.pipeVelX

        # add new pipe when first pipe is about to touch left of screen
        if 0 < self.upperPipes[0]['x'] < 5:
            newPipe = getRandomPipe()
            self.upperPipes.append(newPipe[0])
            self.lowerPipes.append(newPipe[1])

        # remove first pipe if its out of the screen
        if self.upperPipes[0]['x'] < -PIPE_WIDTH:
            self.upperPipes.pop(0)
            self.lowerPipes.pop(0)

        # check if crash here
        isCrash= checkCrash({'x': self.playerx, 'y': self.playery,
                             'index': self.playerIndex},
                            self.upperPipes, self.lowerPipes)
        if isCrash:
            terminal = True
            self.__init__()
            reward = -1

        # draw sprites
        SCREEN.blit(IMAGES['background'], (0,0))

        for uPipe, lPipe in zip(self.upperPipes, self.lowerPipes):
            SCREEN.blit(IMAGES['pipe'][0], (uPipe['x'], uPipe['y']))
            SCREEN.blit(IMAGES['pipe'][1], (lPipe['x'], lPipe['y']))

        SCREEN.blit(IMAGES['base'], (self.basex, BASEY))
        # print score so player overlaps the score
        showScore(self.score)
        SCREEN.blit(IMAGES['player'][self.playerIndex],
                    (self.playerx, self.playery))

        image_data = pygame.surfarray.array3d(pygame.display.get_surface())
        pygame.display.update()
        FPSCLOCK.tick(FPS)
        return image_data, reward, terminal, self.score

def getRandomPipe():
    """returns a randomly generated pipe"""
    # y of gap between upper and lower pipe
    gapYs = [20, 30, 40, 50, 60, 70, 80, 90]
    index = random.randint(0, len(gapYs)-1)
    gapY = gapYs[index]

    gapY += int(BASEY * 0.2)
    pipeX = SCREENWIDTH + 20

    return [
        {'x': pipeX, 'y': gapY - PIPE_HEIGHT},  # upper pipe
        {'x': pipeX, 'y': gapY + PIPEGAPSIZE},  # lower pipe
    ]


def showScore(score):
    """displays score in center of screen"""
    scoreDigits = [int(x) for x in list(str(score))]
    totalWidth = 0 # total width of all numbers to be printed

    for digit in scoreDigits:
        totalWidth += IMAGES['numbers'][digit].get_width()

    Xoffset = (SCREENWIDTH - totalWidth) / 2

    for digit in scoreDigits:
        SCREEN.blit(IMAGES['numbers'][digit], (Xoffset, SCREENHEIGHT * 0.1))
        Xoffset += IMAGES['numbers'][digit].get_width()


def checkCrash(player, upperPipes, lowerPipes):
    """returns True if player collders with base or pipes."""
    pi = player['index']
    player['w'] = IMAGES['player'][0].get_width()
    player['h'] = IMAGES['player'][0].get_height()

    # if player crashes into ground
    if player['y'] + player['h'] >= BASEY - 1:
        return True
    else:

        playerRect = pygame.Rect(player['x'], player['y'],
                      player['w'], player['h'])

        for uPipe, lPipe in zip(upperPipes, lowerPipes):
            # upper and lower pipe rects
            uPipeRect = pygame.Rect(uPipe['x'], uPipe['y'], PIPE_WIDTH, PIPE_HEIGHT)
            lPipeRect = pygame.Rect(lPipe['x'], lPipe['y'], PIPE_WIDTH, PIPE_HEIGHT)

            # player and upper/lower pipe hitmasks
            pHitMask = HITMASKS['player'][pi]
            uHitmask = HITMASKS['pipe'][0]
            lHitmask = HITMASKS['pipe'][1]

            # if bird collided with upipe or lpipe
            uCollide = pixelCollision(playerRect, uPipeRect, pHitMask, uHitmask)
            lCollide = pixelCollision(playerRect, lPipeRect, pHitMask, lHitmask)

            if uCollide or lCollide:
                return True

    return False

def pixelCollision(rect1, rect2, hitmask1, hitmask2):
    """Checks if two objects collide and not just their rects"""
    rect = rect1.clip(rect2)

    if rect.width == 0 or rect.height == 0:
        return False

    x1, y1 = rect.x - rect1.x, rect.y - rect1.y
    x2, y2 = rect.x - rect2.x, rect.y - rect2.y

    for x in range(rect.width):
        for y in range(rect.height):
            if hitmask1[x1+x][y1+y] and hitmask2[x2+x][y2+y]:
                return True
    return False

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


# Now, PyGame and the Flappy Bird Enviornment Is Ready. 

We can now define the hyperparameters that are involved in this experimentation. 

In [3]:
#!/usr/bin/env python
from __future__ import print_function
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import cv2
import sys
import random
import numpy as np
from collections import deque

# seed 1001 was the maximal performance seed. 
seed = 1001
np.random.seed(seed)
random.seed(seed)
tf.compat.v1.set_random_seed(seed)

# if you are running this on Google Colab (e.g., using Google Drive), enable to True. 
drive = False 
google_drive_colab_path = '/content/drive/My Drive/flappy/' if drive == True else ''

OBSERVE = 10000 # timestpes to init the replay memory. 
EXPLORE = 1000000 # frames over which to decay epsilon

FINAL_EPSILON = 0.0001 # final value
INITIAL_EPSILON = 0.4 # starting value  

REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

GAME = 'bird' # the name of the game being played for log files
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations

Instructions for updating:
non-resource variables are not supported in the long term


# Below, you will find the `testing` variable. [[SKIP THIS SECTION IF YOU WOULD LIKE TO RUN THE TRAINING PROCESS, OTHERWISE, YOU MAY ENCOUNTER AN ERROR]]

It's set to true initally. This is becuase, firstly, I will demonstrate the testing process to show reproducibility. Then, I will walk you through the training process.

The network weights for the 130,000,000 frame of the seed 1001 are provided in the saved_networks folder. I didn't want to make the .zip or GitHub too large, so I just provided the best performing weights. Of course, more weights are available, please email me at: aadarsh.jha@vanderbilt.edu

In [4]:

testing = True 

In [5]:

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

def createNetwork():
    # network weights
    weight_conv_1 = weight_variable([8, 8, 4, 32])
    bias_conv_1 = bias_variable([32])

    weight_conv_2 = weight_variable([4, 4, 32, 64])
    bias_conv_2 = bias_variable([64])

    weight_conv_3 = weight_variable([3, 3, 64, 64])
    bias_conv_3 = bias_variable([64])

    weight_fully_1 = weight_variable([1600, 512])
    bias_fully_1 = bias_variable([512])

    weight_fully_2 = weight_variable([512, ACTIONS])
    bias_fully_2 = bias_variable([ACTIONS])

    # input layer
    input_layer = tf.placeholder("float", [None, 80, 80, 4])

    # hidden layers
    hidden_conv_1 = tf.nn.relu(conv2d(input_layer, weight_conv_1, 4) + bias_conv_1)
    pool_1 = max_pool_2x2(hidden_conv_1)

    hidden_conv_2 = tf.nn.relu(conv2d(pool_1, weight_conv_2, 2) + bias_conv_2)
    hidden_conv_3 = tf.nn.relu(conv2d(hidden_conv_2, weight_conv_3, 1) + bias_conv_3)

    hidden_conv_3_flat = tf.reshape(hidden_conv_3, [-1, 1600])

    hidden_fully_connected_1 = tf.nn.relu(tf.matmul(hidden_conv_3_flat, weight_fully_1) + bias_fully_1)

    readout = tf.matmul(hidden_fully_connected_1, weight_fully_2) + bias_fully_2

    return input_layer, readout, hidden_fully_connected_1

def trainNetwork(s, readout, _, sess):
    counter = 0 
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])

    # readout stores the output of the network. 
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices=1)
    cost = tf.reduce_mean(tf.square(y - readout_action))

    # the optimizer is declared. 
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # open up a game state 
    game_state = GameState()

    # store the previous observations in replay memory
    D = deque()

    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1

    # preprocess the image to 80x80x4 and get the image state. 
    x_t, _, terminal, _ = game_state.frame_step(do_nothing)
    x_t = cv2.cvtColor(cv2.resize(x_t, (80, 80)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)

    # saving and loading networks
    saver = tf.train.Saver()
    sess.run(tf.initialize_all_variables())
    checkpoint = tf.train.get_checkpoint_state("saved_networks")

    # are we testing or training? the decision is made here. 
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")

    # start training

    # data structures meant for logging
    epsilon = INITIAL_EPSILON
    t = 0
    score = [] 
    net_score = [] 
    net_flaps = [] 
    flaps = [] 

    # we continue to execute forever, until the game ends.
    while True:

        # get all the actions from the netwokr 
        readout_t = readout.eval(feed_dict={s : [s_t]})[0]

        a_t = np.zeros([ACTIONS])
        action_index = 0

        # if we're testing we dont need to follow an epsilon greedy policy.
        # just get the highest action value. 
        if testing:
            if counter > 10: 
                print("Testing Done")
                return
            if t % FRAME_PER_ACTION == 0:
                action_index = np.argmax(readout_t)
                a_t[action_index] = 1
            else:
                a_t[0] = 1 
        else: 
            # otherwise, we should select randomly at times. (Defined by epsilon)
            if t % FRAME_PER_ACTION == 0:
                if random.random() <= epsilon:
                    print("Random Action Selected Via Epsilon Greedy")
                    action_index = random.randrange(ACTIONS)
                    a_t[random.randrange(ACTIONS)] = 1
                else:
                    action_index = np.argmax(readout_t)
                    a_t[action_index] = 1
            else:
                a_t[0] = 1 # do nothing

        # downscale the value of the epsilon. 
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        x_t1_colored, r_t, terminal, cur_score = game_state.frame_step(a_t)
        flaps.append(cur_score)

        # process the image to 80x80x4 to preparer to feed into the network. 
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored, (80, 80)), cv2.COLOR_BGR2GRAY)
        _, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (80, 80, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        
        score.append(r_t)

        # we store memory via the replay memory.
        if testing == False: 

            # store the transition in D
            D.append((s_t, a_t, r_t, s_t1, terminal))

            # popping when above the memory. 
            if len(D) > REPLAY_MEMORY:
                D.popleft()

            # only train if done observing (We've sufficiently filled the replay memory)
            if t > OBSERVE:
                # sample a minibatch to train on
                minibatch = random.sample(D, BATCH)

                # get the batch variables
                s_j_batch = [d[0] for d in minibatch]
                a_batch = [d[1] for d in minibatch]
                r_batch = [d[2] for d in minibatch]
                s_j1_batch = [d[3] for d in minibatch]

                y_batch = []

                readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
                
                for i in range(0, len(minibatch)):
                    terminal = minibatch[i][4]
                    # if terminal, only equals reward
                    if terminal:
                        y_batch.append(r_batch[i])
                    else:
                        y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

                # perform gradient step
                train_step.run(feed_dict = {
                    y : y_batch,
                    a : a_batch,
                    s : s_j_batch}
                )

        # update the old values
        s_t = s_t1
        t += 1

        if testing == False: 
            # save progress every 10000 iterations
            if t % 10000 == 0:
                saver.save(sess, google_drive_colab_path + 'saved_networks_v1/' + GAME + '-dqn', global_step = t)

            # print info
            state = ""
            if t <= OBSERVE:
                state = "observe"
            elif t > OBSERVE and t <= OBSERVE + EXPLORE:
                state = "explore"
            else:
                state = "train"
                
        if terminal: 
            net_score.append(sum(score))
            net_flaps.append(max(flaps))

        if terminal and testing: 
            counter = counter + 1
            print("TIMESTEP,", t, "Reward,", sum(score), "Average Reward,", np.mean(net_score), "Flaps,", max(flaps), "Average Flaps,", np.mean(net_flaps), "Standard Deviation Flaps,", np.std(net_flaps))
            score = []    
            flaps = [] 

        if terminal and testing == False: 
            string = "GameOver TIMESTEP: " + str(t) + ", STATE: " + str(state) + ", EPSILON: " + str(epsilon) + ", ACTION: " + str(action_index) + ", REWARD: " + str(r_t) + ", Q_MAX: %e" % np.max(readout_t) + ", Episode Reward: " + str(sum(score)) +  ", Average Reward: " + str(np.mean(net_score)) + ", Standard Deviation Of Score: " + str(np.std(net_score)) + ", Flaps: " + str(max(flaps)) +  ", Average Flaps: " + str(np.mean(net_flaps)) + ", Standard Deviation Of Flaps: " + str(np.std(net_flaps))
            print(string)
            print("Game Over")
            with open(google_drive_colab_path + "net_score_cache_v1_game_over.txt", 'a') as f:
                f.write(string + "\n")
                f.close()
            score = [] 
            flaps = [] 

        if terminal == False and testing == False: 
            string = "TIMESTEP: " + str(t) + ", STATE: " + str(state) + ", EPSILON: " + str(epsilon) + ", ACTION: " + str(action_index) + ", REWARD: " + str(r_t) + ", Q_MAX: %e" % np.max(readout_t) + ", Episode Reward: " + str(sum(score)) +  ", Average Reward: " + str(np.mean(net_score)) + ", Standard Deviation Of Score: " + str(np.std(net_score)) + ", Flaps: " + str(max(flaps)) +  ", Average Flaps: " + str(np.mean(net_flaps)) + ", Standard Deviation Of Flaps: " + str(np.std(net_flaps))
            print(string) 

if __name__ == "__main__":
    sess = tf.InteractiveSession()
    input_layer, readout, hidden_fully_connected_1 = createNetwork()
    trainNetwork(input_layer, readout, hidden_fully_connected_1, sess)

2021-12-13 12:38:12.642664: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Restoring parameters from saved_networks/bird-dqn-1300000
Successfully loaded: saved_networks/bird-dqn-1300000
TIMESTEP, 1529 Reward, 186.89999999999495 Average Reward, 186.89999999999495 Flaps, 39 Average Flaps, 39.0 Standard Deviation Flaps, 0.0
TIMESTEP, 3667 Reward, 262.19999999999334 Average Reward, 224.54999999999416 Flaps, 55 Average Flaps, 47.0 Standard Deviation Flaps, 8.0
TIMESTEP, 8465 Reward, 591.2000000000522 Average Reward, 346.76666666668024 Flaps, 125 Average Flaps, 73.0 Standard Deviation Flaps, 37.345236197762446
TIMESTEP, 12732 Reward, 525.5000000000405 Average Reward, 391.4500000000203 Flaps, 111 Average Flaps, 82.5 Standard Deviation Flaps, 36.2870500316573
TIMESTEP, 14338 Reward, 196.39999999999452 Average Reward, 352.4400000000152 Flaps, 41 Average Flaps, 74.2 Standard Deviation Flaps, 36.454903648206226
TIMESTEP, 16249 Reward, 234.09999999999283 Average Reward, 332.71666666

# Above, the testing is executed. 

To interpret this a bit better, here's a summary of the process:
* A log is produced everytime the game is reset (e.g., the bird dies). 
* Reward, and average reward are computed for reference. 
* What is most important to note is the Flaps, Average Flaps, and Standard Deviation of Flaps, as they represent the number of times the bird flaps, which is the primary, and only, metric of how well the user is at playing the game. 

Referencing the paper, one can see how the maximal flaps are: 125, and the average flaps are: 72.0. 

# Now, we can move onto training. 

* We switch the `testing` variable to `True` to enable the training process.
* It is recommended to use a GPU for training. CPU will take roughly 2 days, whereas GPU will take 1 day. 
* The block from above is copied again, since its the same code yet again to train, except we set the `testing` variable to `False`.
* If an error is encountered wherein tensorflow says a session is already in place, simply restart the notebook and do NOT run the testing code. 

In [ ]:
testing = False

In [ ]:

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

def createNetwork():
    # network weights
    weight_conv_1 = weight_variable([8, 8, 4, 32])
    bias_conv_1 = bias_variable([32])

    weight_conv_2 = weight_variable([4, 4, 32, 64])
    bias_conv_2 = bias_variable([64])

    weight_conv_3 = weight_variable([3, 3, 64, 64])
    bias_conv_3 = bias_variable([64])

    weight_fully_1 = weight_variable([1600, 512])
    bias_fully_1 = bias_variable([512])

    weight_fully_2 = weight_variable([512, ACTIONS])
    bias_fully_2 = bias_variable([ACTIONS])

    # input layer
    input_layer = tf.placeholder("float", [None, 80, 80, 4])

    # hidden layers
    hidden_conv_1 = tf.nn.relu(conv2d(input_layer, weight_conv_1, 4) + bias_conv_1)
    pool_1 = max_pool_2x2(hidden_conv_1)

    hidden_conv_2 = tf.nn.relu(conv2d(pool_1, weight_conv_2, 2) + bias_conv_2)
    hidden_conv_3 = tf.nn.relu(conv2d(hidden_conv_2, weight_conv_3, 1) + bias_conv_3)

    hidden_conv_3_flat = tf.reshape(hidden_conv_3, [-1, 1600])

    hidden_fully_connected_1 = tf.nn.relu(tf.matmul(hidden_conv_3_flat, weight_fully_1) + bias_fully_1)

    readout = tf.matmul(hidden_fully_connected_1, weight_fully_2) + bias_fully_2

    return input_layer, readout, hidden_fully_connected_1

def trainNetwork(s, readout, _, sess):
    counter = 0 
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])

    # readout stores the output of the network. 
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices=1)
    cost = tf.reduce_mean(tf.square(y - readout_action))

    # the optimizer is declared. 
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # open up a game state 
    game_state = GameState()

    # store the previous observations in replay memory
    D = deque()

    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1

    # preprocess the image to 80x80x4 and get the image state. 
    x_t, _, terminal, _ = game_state.frame_step(do_nothing)
    x_t = cv2.cvtColor(cv2.resize(x_t, (80, 80)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)

    # saving and loading networks
    saver = tf.train.Saver()
    sess.run(tf.initialize_all_variables())
    checkpoint = tf.train.get_checkpoint_state("saved_networks")

    # are we testing or training? the decision is made here. 
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")

    # start training

    # data structures meant for logging
    epsilon = INITIAL_EPSILON
    t = 0
    score = [] 
    net_score = [] 
    net_flaps = [] 
    flaps = [] 

    # we continue to execute forever, until the game ends.
    while True:

        # get all the actions from the netwokr 
        readout_t = readout.eval(feed_dict={s : [s_t]})[0]

        a_t = np.zeros([ACTIONS])
        action_index = 0

        # if we're testing we dont need to follow an epsilon greedy policy.
        # just get the highest action value. 
        if testing:
            if counter > 10: 
                print("Testing Done")
                return
            if t % FRAME_PER_ACTION == 0:
                action_index = np.argmax(readout_t)
                a_t[action_index] = 1
            else:
                a_t[0] = 1 
        else: 
            # otherwise, we should select randomly at times. (Defined by epsilon)
            if t % FRAME_PER_ACTION == 0:
                if random.random() <= epsilon:
                    print("Random Action Selected Via Epsilon Greedy")
                    action_index = random.randrange(ACTIONS)
                    a_t[random.randrange(ACTIONS)] = 1
                else:
                    action_index = np.argmax(readout_t)
                    a_t[action_index] = 1
            else:
                a_t[0] = 1 # do nothing

        # downscale the value of the epsilon. 
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        x_t1_colored, r_t, terminal, cur_score = game_state.frame_step(a_t)
        flaps.append(cur_score)

        # process the image to 80x80x4 to preparer to feed into the network. 
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored, (80, 80)), cv2.COLOR_BGR2GRAY)
        _, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (80, 80, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        
        score.append(r_t)

        # we store memory via the replay memory.
        if testing == False: 

            # store the transition in D
            D.append((s_t, a_t, r_t, s_t1, terminal))

            # popping when above the memory. 
            if len(D) > REPLAY_MEMORY:
                D.popleft()

            # only train if done observing (We've sufficiently filled the replay memory)
            if t > OBSERVE:
                # sample a minibatch to train on
                minibatch = random.sample(D, BATCH)

                # get the batch variables
                s_j_batch = [d[0] for d in minibatch]
                a_batch = [d[1] for d in minibatch]
                r_batch = [d[2] for d in minibatch]
                s_j1_batch = [d[3] for d in minibatch]

                y_batch = []

                readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
                
                for i in range(0, len(minibatch)):
                    terminal = minibatch[i][4]
                    # if terminal, only equals reward
                    if terminal:
                        y_batch.append(r_batch[i])
                    else:
                        y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

                # perform gradient step
                train_step.run(feed_dict = {
                    y : y_batch,
                    a : a_batch,
                    s : s_j_batch}
                )

        # update the old values
        s_t = s_t1
        t += 1

        if testing == False: 
            # save progress every 10000 iterations
            if t % 10000 == 0:
                saver.save(sess, google_drive_colab_path + 'saved_networks_v1/' + GAME + '-dqn', global_step = t)

            # print info
            state = ""
            if t <= OBSERVE:
                state = "observe"
            elif t > OBSERVE and t <= OBSERVE + EXPLORE:
                state = "explore"
            else:
                state = "train"
                
        if terminal: 
            net_score.append(sum(score))
            net_flaps.append(max(flaps))

        if terminal and testing: 
            counter = counter + 1
            print("TIMESTEP,", t, "Reward,", sum(score), "Average Reward,", np.mean(net_score), "Flaps,", max(flaps), "Average Flaps,", np.mean(net_flaps), "Standard Deviation Flaps,", np.std(net_flaps))
            score = []    
            flaps = [] 

        if terminal and testing == False: 
            string = "GameOver TIMESTEP: " + str(t) + ", STATE: " + str(state) + ", EPSILON: " + str(epsilon) + ", ACTION: " + str(action_index) + ", REWARD: " + str(r_t) + ", Q_MAX: %e" % np.max(readout_t) + ", Episode Reward: " + str(sum(score)) +  ", Average Reward: " + str(np.mean(net_score)) + ", Standard Deviation Of Score: " + str(np.std(net_score)) + ", Flaps: " + str(max(flaps)) +  ", Average Flaps: " + str(np.mean(net_flaps)) + ", Standard Deviation Of Flaps: " + str(np.std(net_flaps))
            print(string)
            print("Game Over")
            with open(google_drive_colab_path + "net_score_cache_v1_game_over.txt", 'a') as f:
                f.write(string + "\n")
                f.close()
            score = [] 
            flaps = [] 

        if terminal == False and testing == False: 
            string = "TIMESTEP: " + str(t) + ", STATE: " + str(state) + ", EPSILON: " + str(epsilon) + ", ACTION: " + str(action_index) + ", REWARD: " + str(r_t) + ", Q_MAX: %e" % np.max(readout_t) + ", Episode Reward: " + str(sum(score)) +  ", Average Reward: " + str(np.mean(net_score)) + ", Standard Deviation Of Score: " + str(np.std(net_score)) + ", Flaps: " + str(max(flaps)) +  ", Average Flaps: " + str(np.mean(net_flaps)) + ", Standard Deviation Of Flaps: " + str(np.std(net_flaps))
            print(string) 

if __name__ == "__main__":
    sess = tf.InteractiveSession()
    input_layer, readout, hidden_fully_connected_1 = createNetwork()
    trainNetwork(input_layer, readout, hidden_fully_connected_1, sess)

# While the training is running...
You will notice the following files being created: 
* `net_score_cache_v1_game_over.txt` -- This contains a log of every each timestep (frame) that is processed. 
* `saved_networks_v1/` -- This contains the checkpointed weights of the network.

The `net_score_cache_v1_game_over.txt` was used for logging. `saved_networks_v1/` was used for testing, as seen above in the notebook. 
**Note `saved_networks_v1/` will note be created until 10,000 frames have been processed.**

Additionally, it is reecommended that you early stop the network at 10,000 (so the above error is normal and fine) for the sake of learning in this enviornment. In reality, this was not stopped so early (rather, 1,300,000 timesteps were ran). This is so you can have some models for yourself to graph and play around with. 

# Now, we can plot. 
* The results of the log are used to plot. Of course, the graphs do not look complete, but are meant as a playground of code rather than a representation of the actual results that was fed from the training process. Logs, graphs, and plots are all avaliable in the `deep-q-learning-results` folder in the root of the repo. 

* Note! You must convert the .txt file to a .csv file before running the below code. 

In [ ]:
# read file with pandas
import pandas as pd
import matplotlib.pyplot as plt

def plot(d1, d2, xlab, ylab, title, file): 
    plt.plot(d1, d2)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.title(title)
    plt.savefig(file)
    plt.close()

# read net_score_cache_game_over
df = pd.read_csv('net_score_cache_v1_game_over.csv')

# the first column is the timestamp
timestamp = df.iloc[:, 0]

#extract number out of timestamp
timestamp_number = []

for element in timestamp: 
    # iterate until the element is a number
    number = ""
    for char in element:
        if char.isdigit(): 
            number += char
    timestamp_number.append(int(number))

#extract average reward
reward = df.iloc[:, 6]
reward_number = []

for element in reward:
    number = "" 
    for i in range(len(element)): 
        if element[i].isdigit() or element[i] == '-': 
            number = element[i:]
            reward_number.append(float(number))
            break

#extract average reward
average_reward = df.iloc[:, 7]
average_reward_number = []

for element in average_reward:
    number = "" 
    for i in range(len(element)): 
        if element[i].isdigit() or element[i] == "-": 
            number = element[i:]
            average_reward_number.append(float(number))
            if (float(number) < 0): 
                print(number)
            break

#extract standard deviation of reward
std_reward = df.iloc[:, 8]
std_reward_number = []

for element in std_reward:
    number = "" 

    for i in range(len(element)): 
        if element[i].isdigit(): 
            number = element[i:]
            std_reward_number.append(float(number))
            break

#plot reward vs timestamp
plot(timestamp_number, reward_number, "Timestamp", "Reward", "Reward vs Timestamp", "reward_vs_timestamp.png")

#plot average_reward_number vs timestamp
plot(timestamp_number, average_reward_number, "Timestamp", "Average Reward", "Average Reward vs Timestamp", "average_reward_vs_timestamp.png")

#plot std_reward_number vs timestamp
plot(timestamp_number, std_reward_number, "Timestamp", "Standard Deviation of Reward", "Standard Deviation of Reward vs Timestamp", "std_reward_vs_timestamp.png")